In [2]:
using JuMP, Cbc

# we'll build the model using number indices
# but we'll display the solution with names
cities = 1:4
states = 1:4
citynames = [:Madison,:Chicago,:DesMoines,:Minneapolis]
statenames = [:Wisconsin,:Illinois,:Iowa,:Minnesota]
cost = [
    15  45  55  70;
    40  10  50  115;
    40  20  65  75;
    65  112 85 10]
demand = [835, 540, 545, 455]
capacity = [1200, 1200, 1200, 1200]
unmet_pen = [180,180,180,180]

fixedcost = 30000

m = Model()

@variable(m, 
    amountsent[cities,states] >= 0)
@variable(m, 
    open[cities], Bin)
@variable(m,
    shortfall[states] >= 0)

@objective(m, Min,
    fixedcost*sum(open[i] for i in cities)
    + sum(cost[i,j]amountsent[i,j] 
        for i in cities for j in states)
    + sum(unmet_pen[j]*shortfall[j] 
        for j in states))

@constraint(m, meetdemand[j in states],
    sum(amountsent[i,j] for i in cities)
       + shortfall[j] == demand[j])

@constraint(m, brewerylimit[i in cities],
    sum(amountsent[i,j] for j in states) 
            <= capacity[i]*open[i])
    
set_optimizer(m, Cbc.Optimizer)
set_silent(m)
optimize!(m)

println("\nTotal cost: \$", 
    objective_value(m))
println("Open brewery in:")
for i in cities
    if value(open[i]) > 0.1
        println(citynames[i])
        for j in states
            if value(amountsent[i,j])>0.001
                println("\tSend ", 
                 round(value(amountsent[i,j]),digits=1), 
                 " barrels to ", statenames[j])
            end
        end
    end
end


Total cost: $139275.0
Open brewery in:
Madison
	Send 745.0 barrels to Wisconsin
	Send 455.0 barrels to Minnesota
Chicago
	Send 90.0 barrels to Wisconsin
	Send 540.0 barrels to Illinois
	Send 545.0 barrels to Iowa
